<a href="https://colab.research.google.com/github/2ndt2nd/Final-Project-IAT-360/blob/main/Final_Project_Jacques_Seth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <b> IAT 360 Final Project <b> </h1></center>

This is the final project for IAT 360
Group Members:
    - Seth van der Zweep (301461888)
    - Jacques Davidson Widodo (301443163)


<h2>Importing required Libraries<h2>

<strong>Import Ultralytics</strong>

Run this one


In [1]:
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

model = YOLO('best.pt')

#CV2 as well
import cv2
cap = cv2.VideoCapture(0)  # Try different indices like 0, 1, 2
if not cap.isOpened():
    print("Error: Could not open webcam.")
else:
    print("Webcam opened successfully!")
    
cap.release()

Ultralytics 8.3.38 🚀 Python-3.12.7 torch-2.5.1+cu124 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Setup complete ✅ (8 CPUs, 19.3 GB RAM, 0.0/9.7 GB disk)
Webcam opened successfully!


<strong>Enum States</strong>
For the states

Run this one


In [2]:
from enum import Enum

class State(Enum):
    START = 0
    ROW_SELECTED = 1
    COLUMN_SELECTED = 2
    CONFIRMING = 3
    DISPENSING = 4

reorder = [3,7,2,5,6,1,4,8]

<strong>For Linux</strong>

Testing video output for Linux (different back end)

In [3]:
'''def process_video_feed(model):
    cap = cv2.VideoCapture(0, cv2.CAP_V4L2)  # Adjust backend for your OS
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 8)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 3.5)
    cap.set(cv2.CAP_PROP_FPS, 10)  # Adjust FPS
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()  # Capture frame
        if not ret:
            print("Error: Failed to grab frame.")
            break

        frame = cv2.resize(frame, (640, 480))
        results = model.predict(frame, conf=0.75)  # YOLO inference
        rendered_frame = results[0].plot()
        cv2.imshow("Webcam Feed", rendered_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()'''

'def process_video_feed(model):\n    cap = cv2.VideoCapture(0, cv2.CAP_V4L2)  # Adjust backend for your OS\n    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 8)\n    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 3.5)\n    cap.set(cv2.CAP_PROP_FPS, 10)  # Adjust FPS\n    \n    if not cap.isOpened():\n        print("Error: Could not open webcam.")\n        return\n\n    while True:\n        ret, frame = cap.read()  # Capture frame\n        if not ret:\n            print("Error: Failed to grab frame.")\n            break\n\n        frame = cv2.resize(frame, (640, 480))\n        results = model.predict(frame, conf=0.75)  # YOLO inference\n        rendered_frame = results[0].plot()\n        cv2.imshow("Webcam Feed", rendered_frame)\n\n        if cv2.waitKey(1) & 0xFF == ord(\'q\'):\n            break\n    \n    cap.release()\n    cv2.destroyAllWindows()'

<strong>For Windows</strong>

Testing video output for windows (different back end)

In [4]:
'''def process_video_feed(model):
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # Adjust backend for your OS
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 8)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 3.5)
    cap.set(cv2.CAP_PROP_FPS, 10)  # Adjust FPS
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()  # Capture frame
        if not ret:
            print("Error: Failed to grab frame.")
            break

        frame = cv2.resize(frame, (640, 480))

        results = model.predict(frame, conf=0.75)  # YOLO inference
        rendered_frame = results[0].plot()
        cv2.imshow("Webcam Feed", rendered_frame)

        class_ids = [int(box.cls[0].item()) for box in results[0].boxes]
        print(class_ids)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    
    
    cap.release()
    cv2.destroyAllWindows()'''

'def process_video_feed(model):\n    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # Adjust backend for your OS\n    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 8)\n    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 3.5)\n    cap.set(cv2.CAP_PROP_FPS, 10)  # Adjust FPS\n    \n    if not cap.isOpened():\n        print("Error: Could not open webcam.")\n        return\n\n    while True:\n        ret, frame = cap.read()  # Capture frame\n        if not ret:\n            print("Error: Failed to grab frame.")\n            break\n\n        frame = cv2.resize(frame, (640, 480))\n\n        results = model.predict(frame, conf=0.75)  # YOLO inference\n        rendered_frame = results[0].plot()\n        cv2.imshow("Webcam Feed", rendered_frame)\n\n        class_ids = [int(box.cls[0].item()) for box in results[0].boxes]\n        print(class_ids)\n\n        if cv2.waitKey(1) & 0xFF == ord(\'q\'):\n            break\n    \n    \n    \n    cap.release()\n    cv2.destroyAllWindows()'

<h2>New In Progress Video Function</h2>

In [5]:
def new_process_video_feed(model, cap):
        ret, frame = cap.read()  # Capture frame
        if not ret:
            print("'''Error: Failed to grab frame.")
            return 0

        #frame = cv2.resize(frame, (640, 480))
        results =   model.predict(frame, conf=0.7)  # YOLO inference
        rendered_frame = results[0].plot()
        cv2.imshow("Webcam Feed", rendered_frame)

        class_ids = [int(box.cls[0].item()) for box in results[0].boxes]

        if(not class_ids): return class_ids
        return reorder[class_ids[0]]

<h2>New In Progress Main</h2>

In [ ]:
def main(model):
    hold = 0
    state = State.START
    #cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # For Windows
    cap = cv2.VideoCapture(0, cv2.CAP_V4L2)  # For Linux
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 16)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 7)
    cap.set(cv2.CAP_PROP_FPS, 30)  # Adjust FPS
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    #Main Loop
    while True:
        gesture = new_process_video_feed(model, cap)
        print(state)
        
        #When no gestures are detected
        if(not gesture):
            hold = 0
        else:
            #If condition to prevent accidentally proceeding
            if(hold<15): hold+=1
            print(gesture)
        
        #Until user shows no gesture by making fist, cannot make another command.
        if(hold==10):
            hold = 100
            
            #If these are gestures for 1-4
            if(gesture<5):
                if(state == State.START):
                    state = State(state.value+1)

                elif(state == State.ROW_SELECTED):
                    state = State(state.value+1)

                elif(state == State.COLUMN_SELECTED):
                    state = State(state.value+1)
                    
                elif(state == State.DISPENSING):
                    state = State.START

            else:
                #If state is greater than 0
                #Change later so you can't cancel when dispensing and further
                if(state.value>0):
                    if(gesture==7):
                        state = State.START
                    elif (gesture==6):
                        state = State(state.value-1)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

<h2>Old Main</h2>
Run this one

In [7]:
main(model)


0: 544x640 (no detections), 150.8ms
Speed: 2.3ms preprocess, 150.8ms inference, 0.8ms postprocess per image at shape (1, 3, 544, 640)
State.START

0: 544x640 (no detections), 151.8ms
Speed: 1.5ms preprocess, 151.8ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)
State.START



Qt: Session management error: Could not open network socket


0: 544x640 (no detections), 144.5ms
Speed: 1.3ms preprocess, 144.5ms inference, 0.4ms postprocess per image at shape (1, 3, 544, 640)
State.START

0: 544x640 (no detections), 143.2ms
Speed: 1.6ms preprocess, 143.2ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)
State.START

0: 544x640 (no detections), 127.6ms
Speed: 1.4ms preprocess, 127.6ms inference, 0.4ms postprocess per image at shape (1, 3, 544, 640)
State.START

0: 544x640 (no detections), 123.4ms
Speed: 1.9ms preprocess, 123.4ms inference, 0.8ms postprocess per image at shape (1, 3, 544, 640)
State.START

0: 544x640 (no detections), 124.7ms
Speed: 1.8ms preprocess, 124.7ms inference, 0.4ms postprocess per image at shape (1, 3, 544, 640)
State.START

0: 544x640 (no detections), 126.5ms
Speed: 1.5ms preprocess, 126.5ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 640)
State.START

0: 544x640 (no detections), 122.1ms
Speed: 1.4ms preprocess, 122.1ms inference, 0.5ms postprocess per image at shape (

<h2>Testing Video Only</h2>

In [8]:
# Start processing the webcam feed
#process_video_feed(model)
#main(model)
